In [1]:
import numpy as np
import lib.datautility as du
import lib.evaluationutility as eu
import lib.tf_network as tfnet
from lib.tf_network2 import Optimizer, Cost, Network, Normalization
import tensorflow as tf
import time
import os
import session_info

c:\users\anthony\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\anthony\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\anthony\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\anth

In [2]:
data, headers = du.read_csv('resources/remnant.csv', max_rows=100)
du.print_descriptives(data, headers)


data, headers = du.read_csv('resources/experimental.csv', max_rows=100)
du.print_descriptives(data, headers)

-- loading resources/remnant.csv...(100%)   
DESCRIPTIVES     100 Rows, 27 Columns   
0 sequence_id_t... (int):      0 missing (0.0%)     mean=5969.00 (SD=0.00)                                                                           
1 student_user_... (int):      0 missing (0.0%)     mean=73142.34 (SD=3621.97)                                                                       
2 target_assign... (int):      0 missing (0.0%)     mean=216684.95 (SD=0.05)                                                                         
3 target_assign... (int):      0 missing (0.0%)     mean=0.59 (SD=0.49)                                                                              
4 prior_assignm... (int):      0 missing (0.0%)     mean=216679.72 (SD=6.26)                                                                         
5 prior_assignm... (string):   0 missing (0.0%)     100 unique values                                                                                
6 prior_assign

In [3]:
key = [1,2,0]
iden = [4]
cov = [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
lab = [[3]]
order = 7

seq = tfnet.format_sequence_from_file('resources/remnant.csv',
                                          key,lab,cov,iden,order)
print('formatting feature columns...')
for i in range(len(seq['x'])):
    for j in range(len(seq['x'][i])):
        try:
            seq['x'][i][j] = np.array(seq['x'][i][j], dtype=np.float32)
        except TypeError:
            print('error...')
            print(seq['x'][i])
            exit(1)
seq['x'] = tfnet.sequence_impute_missing(seq['x'])

print('formatting output labels...')
seq['y'] = tfnet.use_last_multi_label(seq['y'], 0)

print('done!')

-- loading resources/remnant.csv...(100%)   
-- preparing to format sequence from file...
-- formatting sequence...(0.32%) 

C:\Users\Anthony\Projects\Rebar\Replication\Study_2\lib\tf_network.py:2000: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  seq['x'] = np.array(x)
C:\Users\Anthony\Projects\Rebar\Replication\Study_2\lib\tf_network.py:2002: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  seq['iden'] = np.array(id)


-- formatting sequence...(100%)   


c:\users\anthony\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


formatting feature columns...
formatting output labels...
done!


In [4]:
key = [1,2,0]
iden = [4]
cov = [10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26]
lab = [[3]]
order = 7

seqex = tfnet.format_sequence_from_file('resources/experimental.csv',
                                          key,lab,cov,iden,order)
print('formatting feature columns...')
for i in range(len(seqex['x'])):
    for j in range(len(seqex['x'][i])):
        try:
            seqex['x'][i][j] = np.array(seqex['x'][i][j], dtype=np.float32)
        except TypeError:
            print('error...')
            print(seqex['x'][i])
            exit(1)
seqex['x'] = tfnet.sequence_impute_missing(seqex['x'])

print('formatting output labels...')
seqex['y'] = tfnet.use_last_multi_label(seqex['y'], 0)

print('done!')

-- loading resources/experimental.csv...(100%)   
-- preparing to format sequence from file...
-- formatting sequence...(100%)   
formatting feature columns...
formatting output labels...
done!


In [5]:
n_cov = len(seq['x'][0][0])

max_epochs = 200
use_validation = True
hidden = [50]
batch = [64]
layers = [1]
keep = [.5]
step = [1e-3]
perf = []
threshold = [0.0001]
optimizer = [Optimizer.ADAM]

tfnet.describe_multi_label(seq['y'], True)

======  Label Description  =============
-- Number of Samples: 129835
-- Number of Label Sets: 1
---- 1: 1 Label



{'n_samples': 129835, 'n_label_sets': 1, 'n_labels': [1]}

In [6]:
tf.reset_default_graph()
tf.set_random_seed(1)
np.random.seed(1)

net = Network('study2_model').add_input_layer(n_cov, normalization=Normalization.Z_SCORE)
net.add_lstm_layer(hidden[0], peepholes=True)

net.begin_multi_output(cost_methods=[Cost.BINARY_CROSS_ENTROPY])
net.add_dropout_layer(1, keep=keep[0], activation=tf.nn.sigmoid)
net.end_multi_output()

net.set_default_cost_method(Cost.CROSS_ENTROPY)
net.set_optimizer(optimizer[0])

net.train(x=seq['x'],
          y=seq['y'],
          step=step[0],
          use_validation=True,
          max_epochs=max_epochs, threshold=threshold[0], batch=batch[0])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
WARNING - Incomplete validation set information was provided. Using default 70/30% split.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

            Training Network            
-3 layers: 17n -> 50n -> 1n
-200 epochs
-step size = 0.001
-batch size = 64
Epoch        Cost    Val Cost   Delta         Time


C:\Users\Anthony\Projects\Rebar\Replication\Study_2\lib\tf_network.py:1047: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  np.array([a[i] for a in series_label])]).reshape((len(series_batch), n_timestep, -1)))


Epoch 1     0.3912    0.2660    0.0000      119.6s
Epoch 2     0.3699    0.2648    0.0006      120.4s
Epoch 3     0.3664    0.2631    0.0008      100.3s
Epoch 4     0.3650    0.2615    0.0008      100.7s
Epoch 5     0.3640    0.2602    0.0007       99.6s
Epoch 6     0.3627    0.2600    0.0012      100.6s
Epoch 7     0.3617    0.2594    0.0011      100.6s
Epoch 8     0.3607    0.2602    0.0006      101.0s
Epoch 9     0.3601    0.2608    0.0001      100.4s
Epoch 10    0.3595    0.2602    0.0000      100.6s
Total Time: 1043.8s


In [7]:
net.save_model_weights()      # comment to use a previously-saved model
#
# net.build()                 # uncomment to use previously-saved model
# net.restore_model_weights() # uncomment to use previously-saved model

In [8]:
print('writing predictions to file...')
du.write_csv(tfnet.flatten_sequence(np.array([np.array([t[-1]]) for t in net.predict(seqex['x'])[0]]),
                                        seqex['key'].reshape((-1,3))), 'model_predictions.csv',
             ['user_id','target_assignment_id','target_sequence_id','pcomplete'])
print('done!')

writing predictions to file...


C:\Users\Anthony\Projects\Rebar\Replication\Study_2\lib\tf_network.py:1427: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  pred.append(np.array(out_p))


done!


In [ ]:
import graph_def_editor 
import warnings
import time
import csv
import sys
import os
import pandas 
from sklearn.metrics import f1_score, r2_score
from sklearn.metrics import mean_squared_error
from skll.metrics import kappa as kpa
import pickle
import psycopg2 as pg
import functools
import Levenshtein as lev
from nltk.tokenize import word_tokenize

session_info.show()

In [9]:
net.session.close()